In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import pickle

In [2]:
from PIL import Image

In [3]:
names = open("image_names.txt", "w")
image_names = os.listdir('../data/ALLSTIMULI/')[2:-3]
for i in range(len(image_names)):
    names.write(image_names[i][:-5]+'\n')
names.close()

In [4]:
names = open("image_names.txt", "r")
img_names = names.readlines()
for i in range(len(img_names)):
    img_names[i]=img_names[i][:-1]
    
loc_data_xy={}
for name in img_names:
    locpath = '../data/loc_data/' + name
    f = open(locpath,'rb')
    loc_dict = pickle.load(f)
    loc_data_xy[name] = np.array(loc_dict['barycenters'])

In [5]:
def show_landmarks(image, landmarks):
    """Show image with landmarks"""
    plt.imshow(image)
    plt.scatter(landmarks[:, 0], landmarks[:, 1], s=10, marker='.', c='r')
    #plt.pause(0.001)  # pause a bit so that plots are updated

# Dataset class

In [6]:
class SaccadeLandmarksDataset(Dataset):
    """Saccade Landmarks dataset."""

    def __init__(self, loc_dict, img_dir, transform=None):
        """
        Args:
            loc_dir (string): Path to the saccade location file
            img_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.loc_dict = loc_dict
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.loc_dict)

    def __getitem__(self, idx):

        img_name = os.listdir(self.img_dir)[idx+2]
        img_path = os.path.join(self.img_dir,img_name)
        image = io.imread(img_path)
        name = img_name[:-5]
        landmarks = self.loc_dict[name]
        landmarks = np.array([landmarks])
        landmarks = landmarks.reshape(-1, 2) #.astype('float').reshape(-1, 2)
        sample = {'image': image, 'landmarks': landmarks}

        if self.transform:
            sample = self.transform(sample)

        return sample

# Transforms

In [7]:
class RandomSaccadeTo(object):
    def __call__(self, sample):
        image, landmarks = sample['image'], sample['landmarks']
        nb_sac = len(landmarks)
        sac_num =  np.random.randint(nb_sac)
        sac = landmarks[sac_num]
        N_X, N_Y = image.shape[:2]
        #img_color_sac = saccade_to(image, (N_X//2, N_Y//2), (sac[1], sac[0]))
        image_roll = np.copy(image)
        image_roll=np.roll(image_roll, N_X//2 - sac[1], axis=0)
        image_roll=np.roll(image_roll, N_Y//2 - sac[0], axis=1)
        return {'image':image_roll, 'pos':sac}

In [8]:
class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):

        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        image_tens = sample['image'].transpose((2, 0, 1))
        return {'image': torch.FloatTensor(image_tens), 'pos': sample['pos']}

### Adapted cropped pyramid (squeezed tensor)

In [9]:
from PYramid import cropped_pyramid

lg shape= 32 32


In [10]:
class CroppedPyramid(object):
    def __init__(self, width, base_levels, color=True, do_mask=False, verbose=False):
        self.width = width
        self.base_levels = base_levels
        self.color = color
        self.do_mask = do_mask
        self.verbose = verbose
    
    def __call__(self, sample):
        img_crop, level_size = cropped_pyramid(sample['image'].unsqueeze(0), 
                                               width=self.width, 
                                               base_levels=self.base_levels,
                                               color=self.color, 
                                               do_mask=self.do_mask, 
                                               verbose=self.verbose,
                                               squeeze=True)
        return{'img_crop':img_crop, 'level_size':level_size, 'pos':sample['pos']}
        
    

### LogGaborTransform

In [12]:
width=32
base_levels=2

In [13]:
from LogGabor import LogGabor
from PYramid import local_filter
from PYramid import get_K
from PYramid import log_gabor_transform

In [16]:
n_sublevel = 2 
n_azimuth = 12 
n_theta = 12
n_phase = 2

pe = {'N_X': width, 'N_Y': width, 'do_mask': False, 'base_levels':
          base_levels, 'n_theta': 24, 'B_sf': 0.6, 'B_theta': np.pi/12 ,
      'use_cache': True, 'figpath': 'results', 'edgefigpath':
          'results/edges', 'matpath': 'cache_dir', 'edgematpath':
          'cache_dir/edges', 'datapath': 'database/', 'ext': '.pdf', 'figsize':
          14.0, 'formats': ['pdf', 'png', 'jpg'], 'dpi': 450, 'verbose': 0}   

lg = LogGabor(pe)
K = get_K(width=width,
            n_sublevel = n_sublevel, 
          n_azimuth = n_azimuth, 
                  n_theta = n_theta,
                  n_phase = n_phase, 
                  r_min = width/6, 
                  r_max = width/3, 
                  log_density_ratio = 2, 
                  verbose=False)

In [17]:
class LogGaborTransform(object):
    def __init__(self, K=K, color=True, verbose=False):
        self.K = K
        self.color = color
        self.verbose = verbose
    
    def __call__(self, sample):
        log_gabor_coeffs = log_gabor_transform(sample['img_crop'].unsqueeze(0), K)
        
        return{'img_gabor':log_gabor_coeffs, 'K':K}

### ComplexModulus

# Compose transforms
### transforms.Compose

# Iterating through the dataset

In [18]:
# Helper function to show a batch
def show_landmarks_batch(sample_batched):
    """Show image with landmarks for a batch of samples."""
    for level in range(5,0,-1):
        plt.figure()
        images_batch = sample_batched['img_crop'][:,level,:,:,:]
        batch_size = len(images_batch)
        im_size = images_batch.size(2)
        grid_border_size = 2

        grid = utils.make_grid(images_batch)
        plt.imshow(grid.numpy().transpose((1, 2, 0)).clip(0,255).astype('uint8'))

        plt.title('Batch from dataloader, level=' + str(level))


In [22]:
N_X = width
N_Y = width
n_sublevel = 2 
n_azimuth = 12 
n_theta = 12
n_phase = 2
n_levels = int(np.log(np.max((N_X, N_Y))/width)/np.log(base_levels)) + 1
n_eccentricity = 2
n_color = 3


In [23]:
class AutoEncoder(nn.Module):
    def __init__(self, in_chan = n_levels * n_color * n_eccentricity * n_azimuth * n_theta * n_phase, out_chan = 100):
        super(AutoEncoder, self).__init__()
        self.encoder = Encoder(in_chan=in_chan, out_chan=out_chan)
        self.decoder = Decoder(in_chan=out_chan, out_chan=in_chan)

    def forward(self, x, **kargs):
        code = self.encoder(x)
        out = self.decoder(code)
        return out

In [24]:
class Encoder(nn.Module):
    """ Encoder
    """
    def __init__(self, in_chan, out_chan):
        super(Encoder, self).__init__()
        self.enc = nn.Linear(in_chan, out_chan)
            
    def forward(self, x):
        return self.enc(x)

In [25]:
class Decoder(nn.Module):
    """ Decoder
    """
    def __init__(self, in_chan, out_chan):
        super(Decoder, self).__init__()
        self.dec = nn.Linear(in_chan, out_chan)
            
    def forward(self, x):
        return self.dec(x)

In [26]:
autoenc = AutoEncoder()

In [27]:
autoenc.encoder.enc

Linear(in_features=1728, out_features=100, bias=True)

In [28]:
autoenc.encoder.enc.weight

Parameter containing:
tensor([[ 0.0048,  0.0118, -0.0006,  ..., -0.0165, -0.0185, -0.0056],
        [ 0.0006, -0.0073, -0.0144,  ...,  0.0162, -0.0165,  0.0097],
        [ 0.0227, -0.0181,  0.0221,  ...,  0.0145,  0.0169,  0.0100],
        ...,
        [-0.0101,  0.0035,  0.0027,  ...,  0.0071,  0.0223, -0.0039],
        [-0.0167, -0.0140,  0.0129,  ..., -0.0100,  0.0236,  0.0061],
        [-0.0168, -0.0221,  0.0037,  ..., -0.0078, -0.0206,  0.0011]],
       requires_grad=True)

In [29]:
autoenc.decoder.dec

Linear(in_features=100, out_features=1728, bias=True)

In [30]:
criterion = nn.MSELoss() 

In [31]:
optimizer = optim.Adam(autoenc.parameters())

In [32]:
batch_size=15


In [33]:
dataloader = DataLoader(saccade_dataset, batch_size=batch_size,
                        shuffle=True, num_workers=0)

NameError: name 'saccade_dataset' is not defined

In [34]:
sample_batched['img_crop'] #input

NameError: name 'sample_batched' is not defined